# PreProcess csv files from CellProfiler

We built a pipeline to segment red blood cells (RBC), nuclei (based on hematoxylin), and nucleated cells (based on eosin).  
* CP_20220429f = Ypos  
* CP_20220429g = Yneg  

We used MeasureShape and MeasureNeighbor. Thus, we get about 68 measures per object (spreadsheet columns), for 4 object classes (csv files), for as many objects as were detected in each path (spreadsheet rows).  

    39616 Cells.csv 68 columns
    39616 Nuclei.csv 68 columns
    60887 RBC.csv 60 columns
    12980 Image.csv 62 columns
    153099 total

Here, each image is actually a patch. The image data includes redundant fields: #RBC, #nuclei, #cells. But the rest of the image data seems non-redundant, including thresholds used and total areas covered. CellProfiler assigned image numbers in order starting at 1 for the first patch. 

The Images.csv contains fields called Filename and URL that contain the path and filename of the patch. For example, D:Martinez/B3.2.jpg is patch 2 of tumor B3.

We ran CellProfiler on the full training set (excludes a 20% test set) of center patches. Here, transform per-object metrics into per-patch metrics. Also, exclude patches with no nuclei.

In [1]:
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn
print('sklearn',sklearn.__version__)

Python 3.8.10
sklearn 1.0.2


In [2]:
YPOS_DIR='/home/jrm/Martinez/CellProfilerRuns/CP_20220429f/'
YNEG_DIR='/home/jrm/Martinez/CellProfilerRuns/CP_20220429g/'
IMAGES="Image.csv"
REDS="RBC.csv"
NUKES="Nuclei.csv"
CELLS="Cells.csv"
MIN_NUCLEI = 2

In [3]:
def describe_all_columns(df):
    count1 = df.isnull().sum().sum()
    print('This many NaN:', count1)
    with pd.option_context('display.max_columns', None):
        print(df.describe(include='all'))
def make_dataframe(filename):
    df = pd.read_csv(filename) 
    return df
def shave_images(df):
    # drop uninformative columns (mostly just zero)
    bad_columns = ['ProcessingStatus', 'Height_HE', 'Width_HE', 'Scaling_HE']
    bad_columns += ['Series_HE', 'Frame_HE', 'Channel_HE', 'Group_Number']
    df = df.drop(columns=bad_columns)
    # drop strings, like filepaths, that could leak the Ypos/Yneg labels
    df = df.select_dtypes(['number'])
    # drop CellProfiler timing, errors (should all be zero), and metadata stats
    df = df.drop(df.filter(regex='^Metadata_').columns, axis=1)
    df = df.drop(df.filter(regex='^ExecutionTime_').columns, axis=1)
    df = df.drop(df.filter(regex='^ModuleError_').columns, axis=1)
    df = df[df['Count_Nuclei']>=MIN_NUCLEI]
    df = df.fillna(0)
    return df


In [4]:
filename = YPOS_DIR+IMAGES
df_ipos = make_dataframe(filename) 
df_ipos = shave_images(df_ipos)
#describe_all_columns(df_ipos)
df_ipos.describe()

,Count_Cells,Count_Nuclei,Count_RBC,Group_Index,ImageNumber,Threshold_FinalThreshold_Cells,Threshold_FinalThreshold_Nuclei,Threshold_FinalThreshold_RBC,Threshold_GuideThreshold_Cells,Threshold_GuideThreshold_Nuclei,Threshold_OrigThreshold_Cells,Threshold_OrigThreshold_Nuclei,Threshold_OrigThreshold_RBC,Threshold_SumOfEntropies_Cells,Threshold_SumOfEntropies_Nuclei,Threshold_SumOfEntropies_RBC,Threshold_WeightedVariance_Cells,Threshold_WeightedVariance_Nuclei,Threshold_WeightedVariance_RBC
count,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000
mean,3.699343,3.699343,4.346731,6446.005098,6446.005098,0.198198,0.229849,0.447377,0.190327,0.231883,0.200201,0.229477,0.372814,-11.732075,-11.436286,-10.138482,0.948547,0.679775,2.199110
std,1.478569,1.478569,3.673069,3889.139278,3889.139278,0.070158,0.051438,0.090652,0.069516,0.057026,0.070901,0.050200,0.075544,0.797363,0.926382,0.882962,0.462473,0.577341,1.485949
min,2.000000,2.000000,0.000000,1.000000,1.000000,0.000166,0.026288,0.000304,0.000165,0.022708,0.000166,0.024872,0.000254,-13.905191,-13.968753,-13.900173,0.008287,0.002667,0.034349
25%,2.000000,2.000000,2.000000,2878.000000,2878.000000,0.162104,0.196336,0.406246,0.153097,0.192319,0.165786,0.196755,0.338538,-12.323464,-11.899810,-10.604216,0.594257,0.285706,1.021031
50%,3.000000,3.000000,4.000000,6650.000000,6650.000000,0.197002,0.234884,0.452503,0.189919,0.242477,0.199161,0.232378,0.377086,-11.903494,-11.494668,-10.201425,0.948817,0.487546,1.873390
75%,5.000000,5.000000,6.000000,9924.000000,9924.000000,0.234956,0.267813,0.498460,0.226771,0.275309,0.237304,0.266793,0.415383,-11.255853,-11.144000,-9.724634,1.288512,0.880290,3.134938
max,10.000000,10.000000,41.000000,12979.000000,12979.000000,0.513549,0.590891,0.707806,0.526853,0.554902,0.513549,0.601256,0.589838,-7.785976,-5.882243,0.000000,2.795326,3.681442,9.340085


In [6]:
df_ineg = make_dataframe(filename) 
df_ineg = shave_images(df_ineg)
#describe_all_columns(df_ineg)
df_ineg.describe()

,Count_Cells,Count_Nuclei,Count_RBC,Group_Index,ImageNumber,Threshold_FinalThreshold_Cells,Threshold_FinalThreshold_Nuclei,Threshold_FinalThreshold_RBC,Threshold_GuideThreshold_Cells,Threshold_GuideThreshold_Nuclei,Threshold_OrigThreshold_Cells,Threshold_OrigThreshold_Nuclei,Threshold_OrigThreshold_RBC,Threshold_SumOfEntropies_Cells,Threshold_SumOfEntropies_Nuclei,Threshold_SumOfEntropies_RBC,Threshold_WeightedVariance_Cells,Threshold_WeightedVariance_Nuclei,Threshold_WeightedVariance_RBC
count,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000,10201.000000
mean,3.699343,3.699343,4.346731,6446.005098,6446.005098,0.198198,0.229849,0.447377,0.190327,0.231883,0.200201,0.229477,0.372814,-11.732075,-11.436286,-10.138482,0.948547,0.679775,2.199110
std,1.478569,1.478569,3.673069,3889.139278,3889.139278,0.070158,0.051438,0.090652,0.069516,0.057026,0.070901,0.050200,0.075544,0.797363,0.926382,0.882962,0.462473,0.577341,1.485949
min,2.000000,2.000000,0.000000,1.000000,1.000000,0.000166,0.026288,0.000304,0.000165,0.022708,0.000166,0.024872,0.000254,-13.905191,-13.968753,-13.900173,0.008287,0.002667,0.034349
25%,2.000000,2.000000,2.000000,2878.000000,2878.000000,0.162104,0.196336,0.406246,0.153097,0.192319,0.165786,0.196755,0.338538,-12.323464,-11.899810,-10.604216,0.594257,0.285706,1.021031
50%,3.000000,3.000000,4.000000,6650.000000,6650.000000,0.197002,0.234884,0.452503,0.189919,0.242477,0.199161,0.232378,0.377086,-11.903494,-11.494668,-10.201425,0.948817,0.487546,1.873390
75%,5.000000,5.000000,6.000000,9924.000000,9924.000000,0.234956,0.267813,0.498460,0.226771,0.275309,0.237304,0.266793,0.415383,-11.255853,-11.144000,-9.724634,1.288512,0.880290,3.134938
max,10.000000,10.000000,41.000000,12979.000000,12979.000000,0.513549,0.590891,0.707806,0.526853,0.554902,0.513549,0.601256,0.589838,-7.785976,-5.882243,0.000000,2.795326,3.681442,9.340085


Aggregator [tutorial](https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/).
Python built-in functions [list](https://docs.python.org/3/library/functions.html).


In [46]:
def shave_objects(df,prefix):
    bad_columns=['Location_Center_X','Location_Center_Y','Location_Center_Z','Number_Object_Number']
    df = df.drop(columns=bad_columns)
    renames = {}
    for col in df.columns.values.tolist():
        renames[col]=prefix+col
    del renames['ImageNumber']
    df.rename(columns=renames,inplace=True)
    agg_cols = {}
    for col in df.columns.values.tolist():
        agg_cols[col]=[min,max,np.mean,np.std]
    del agg_cols['ImageNumber']
    agg_cols[prefix+'ObjectNumber']=[max]
    df = df[df['ImageNumber']>0].groupby('ImageNumber').agg(agg_cols)
    df = df.fillna(0)
    return df

In [48]:
filename = YPOS_DIR+REDS
df_rpos = make_dataframe(filename) 
df_rpos = shave_objects(df_rpos,"RBC_")
df_rpos.head()

R_ObjectNumber R_AreaShape_Area                                \
                       max              min   max        mean         std   
ImageNumber                                                                 
1                        5              335   587  405.800000  102.726335   
2                        7              368   609  465.857143  103.246354   
3                        7              360  1214  670.285714  325.690914   
4                        2              432   508  470.000000   53.740115   
5                        6              389  1438  908.166667  459.363654   

            R_AreaShape_BoundingBoxArea                                 \
                                    min   max         mean         std   
ImageNumber                                                              
1                                   533  1178   766.400000  278.347445   
2                                   616   990   830.571429  157.656649   
3                                   558  2920  1523.571429  893.488268   
4                                   696  1075   885.500000  267.993470   
5                                   672  3038  1625.666667  941.521676   

            R_AreaShape_BoundingBoxMaximum_X  ... R_AreaShape_Zernike_9_5  \
                                         min  ...                    mean   
ImageNumber                                   ...                           
1                                         28  ...                0.005144   
2                                         54  ...                0.004510   
3                                         34  ...                0.007529   
4                                        103  ...                0.006922   
5                                         45  ...                0.005835   

                      R_AreaShape_Zernike_9_7                                \
                  std                     min       max      mean       std   
ImageNumber                                                                   
1            0.002340                0.004830  0.011568  0.007626  0.002476   
2            0.002460                0.002937  0.011398  0.005770  0.002833   
3            0.003319                0.003413  0.019292  0.008709  0.005355   
4            0.000935                0.001533  0.005840  0.003686  0.003046   
5            0.001545                0.000291  0.013358  0.005866  0.004328   

            R_AreaShape_Zernike_9_9                                
                                min       max      mean       std  
ImageNumber                                                        
1                          0.001434  0.004422  0.002700  0.001349  
2                          0.001007  0.014364  0.006812  0.004692  
3                          0.002975  0.009223  0.005838  0.002058  
4                          0.000358  0.005038  0.002698  0.003309  
5                          0.000613  0.007169  0.003813  0.002628  

[5 rows x 221 columns]

In [49]:
filename = YNEG_DIR+REDS
df_rneg = make_dataframe(filename) 
df_rneg = shave_objects(df_rneg,"RBC_")
df_rneg.head()

RBC_ObjectNumber RBC_AreaShape_Area                                \
                         max                min   max        mean         std   
ImageNumber                                                                     
1                          1                326   326  326.000000    0.000000   
2                          1                656   656  656.000000    0.000000   
3                          3                321  1373  850.666667  526.038338   
4                          3                384   957  622.000000  298.561551   
5                          4                442   921  643.000000  210.287105   

            RBC_AreaShape_BoundingBoxArea                                  \
                                      min   max         mean          std   
ImageNumber                                                                 
1                                     444   444   444.000000     0.000000   
2                                    1170  1170  1170.000000     0.000000   
3                                     432  3015  1649.000000  1297.930275   
4                                     720  2268  1313.333333   834.863662   
5                                     756  1440  1063.500000   338.413455   

            RBC_AreaShape_BoundingBoxMaximum_X  ... RBC_AreaShape_Zernike_9_5  \
                                           min  ...                      mean   
ImageNumber                                     ...                             
1                                          224  ...                  0.005872   
2                                          213  ...                  0.001733   
3                                           12  ...                  0.006390   
4                                           18  ...                  0.009179   
5                                           86  ...                  0.007222   

                      RBC_AreaShape_Zernike_9_7                                \
                  std                       min       max      mean       std   
ImageNumber                                                                     
1            0.000000                  0.005488  0.005488  0.005488  0.000000   
2            0.000000                  0.007225  0.007225  0.007225  0.000000   
3            0.002821                  0.002655  0.007320  0.004739  0.002372   
4            0.006109                  0.006217  0.007172  0.006749  0.000486   
5            0.003745                  0.003436  0.011464  0.006879  0.003397   

            RBC_AreaShape_Zernike_9_9                                
                                  min       max      mean       std  
ImageNumber                                                          
1                            0.006943  0.006943  0.006943  0.000000  
2                            0.003105  0.003105  0.003105  0.000000  
3                            0.001502  0.003882  0.002320  0.001353  
4                            0.003527  0.004779  0.004243  0.000645  
5                            0.002548  0.005582  0.004592  0.001391  

[5 rows x 221 columns]

In [51]:
filename = YPOS_DIR+NUKES
df_npos = make_dataframe(filename) 
df_npos = shave_objects(df_npos,"Nuc_")
df_npos.head()

Nuc_ObjectNumber Nuc_AreaShape_Area                     \
                         max                min   max         mean   
ImageNumber                                                          
1                          2               2347  2355  2351.000000   
2                          2               3046  3080  3063.000000   
3                          1               4413  4413  4413.000000   
4                          3               1546  2747  2024.333333   
5                          3               1779  4818  3488.333333   

                         Nuc_AreaShape_BoundingBoxArea                       \
                     std                           min    max          mean   
ImageNumber                                                                   
1               5.656854                          4672   7371   6021.500000   
2              24.041631                          5472  14960  10216.000000   
3               0.000000                         10700  10700  10700.000000   
4             636.690139                          3015   4980   3766.333333   
5            1554.667274                          4774   9165   7619.000000   

                         Nuc_AreaShape_BoundingBoxMaximum_X  ...  \
                     std                                min  ...   
ImageNumber                                                  ...   
1            1908.481202                                133  ...   
2            6709.029140                                 78  ...   
3               0.000000                                219  ...   
4            1060.952559                                 62  ...   
5            2466.935548                                200  ...   

            Nuc_Neighbors_PercentTouching_Expanded             \
                                              mean        std   
ImageNumber                                                     
1                                        45.921546   5.295771   
2                                        52.610673  17.147118   
3                                        72.211350   0.000000   
4                                        61.615385  15.230768   
5                                        60.974195  20.051846   

            Nuc_Neighbors_SecondClosestDistance_Expanded              \
                                                     min         max   
ImageNumber                                                            
1                                               0.000000    0.000000   
2                                             131.280259  134.728636   
3                                             104.051405  104.051405   
4                                             101.280999  112.014678   
5                                             134.643971  163.484110   

                                    \
                   mean        std   
ImageNumber                          
1              0.000000   0.000000   
2            133.004448   2.438371   
3            104.051405   0.000000   
4            106.935655   5.389943   
5            153.870731  16.650862   

            Nuc_Neighbors_SecondClosestObjectNumber_Expanded                \
                                                         min max      mean   
ImageNumber                                                                  
1                                                          0   0  0.000000   
2                                                          0   0  0.000000   
3                                                          0   0  0.000000   
4                                                          1   3  1.666667   
5                                                          1   3  2.333333   

                       
                  std  
ImageNumber            
1            0.000000  
2            0.000000  
3            0.000000  
4            1.154701  
5            1.154701  

[5 rows x 253 columns]

In [53]:
filename = YNEG_DIR+NUKES
df_nneg = make_dataframe(filename) 
df_nneg = shave_objects(df_nneg,"Nuc_")
df_nneg.head()

Nuc_ObjectNumber Nuc_AreaShape_Area                             \
                         max                min   max    mean          std   
ImageNumber                                                                  
1                          2               1477  1480  1478.5     2.121320   
2                          2               1381  2053  1717.0   475.175757   
3                          1               2508  2508  2508.0     0.000000   
4                          6               1518  4894  3513.0  1464.860130   
5                          2               2937  3997  3467.0   749.533188   

            Nuc_AreaShape_BoundingBoxArea                              \
                                      min    max    mean          std   
ImageNumber                                                             
1                                    3000   3640  3320.0   452.548340   
2                                    2862   3519  3190.5   464.569155   
3                                    7455   7455  7455.0     0.000000   
4                                    3294  16999  7797.5  4917.225712   
5                                    5822   7623  6722.5  1273.499313   

            Nuc_AreaShape_BoundingBoxMaximum_X  ...  \
                                           min  ...   
ImageNumber                                     ...   
1                                          154  ...   
2                                          170  ...   
3                                           99  ...   
4                                           92  ...   
5                                           71  ...   

            Nuc_Neighbors_PercentTouching_Expanded             \
                                              mean        std   
ImageNumber                                                     
1                                        42.833004  10.475074   
2                                        40.765766  13.059179   
3                                         0.000000   0.000000   
4                                        73.899362  16.731575   
5                                        60.901186   1.419270   

            Nuc_Neighbors_SecondClosestDistance_Expanded              \
                                                     min         max   
ImageNumber                                                            
1                                               0.000000    0.000000   
2                                               0.000000    0.000000   
3                                               0.000000    0.000000   
4                                              59.598669  120.101924   
5                                             150.856407  150.856407   

                                    \
                   mean        std   
ImageNumber                          
1              0.000000   0.000000   
2              0.000000   0.000000   
3              0.000000   0.000000   
4             85.878868  21.757289   
5            150.856407   0.000000   

            Nuc_Neighbors_SecondClosestObjectNumber_Expanded           \
                                                         min max mean   
ImageNumber                                                             
1                                                          0   0  0.0   
2                                                          0   0  0.0   
3                                                          0   0  0.0   
4                                                          3   6  4.5   
5                                                          0   0  0.0   

                       
                  std  
ImageNumber            
1            0.000000  
2            0.000000  
3            0.000000  
4            1.643168  
5            0.000000  

[5 rows x 253 columns]

In [54]:
filename = YPOS_DIR+CELLS
df_cpos = make_dataframe(filename) 
df_cpos = shave_objects(df_cpos,"Cell_")
df_cpos.head()

Cell_ObjectNumber Cell_AreaShape_Area                  \
                          max                 min    max     mean   
ImageNumber                                                         
1                           2                4750   8617   6683.5   
2                           2                5986  14025  10005.5   
3                           1                6736   6736   6736.0   
4                           3                2925  17223   7750.0   
5                           3                5435   7352   6292.0   

                         Cell_AreaShape_BoundingBoxArea                       \
                     std                            min    max          mean   
ImageNumber                                                                    
1            2734.381923                           8505  15080  11792.500000   
2            5684.431414                           9894  27520  18707.000000   
3               0.000000                          12876  12876  12876.000000   
4            8204.335988                           6300  27984  13756.000000   
5             974.489097                          11020  23490  16378.666667   

                          Cell_AreaShape_BoundingBoxMaximum_X  ...  \
                      std                                 min  ...   
ImageNumber                                                    ...   
1             4649.227086                                 140  ...   
2            12463.464125                                 102  ...   
3                0.000000                                 224  ...   
4            12326.554750                                  84  ...   
5             6417.094774                                 200  ...   

            Cell_Neighbors_SecondClosestDistance_Expanded             \
                                                     mean        std   
ImageNumber                                                            
1                                               58.006418   6.755942   
2                                               71.603242   6.743078   
3                                               60.697177   0.000000   
4                                               58.321133  26.255213   
5                                               66.434070   9.318718   

            Cell_Neighbors_SecondClosestObjectNumber_Expanded                \
                                                          min max      mean   
ImageNumber                                                                   
1                                                           0   0  0.000000   
2                                                           0   0  0.000000   
3                                                           0   0  0.000000   
4                                                           1   2  1.333333   
5                                                           1   3  2.333333   

                      Cell_Parent_Nuclei                     
                  std                min max mean       std  
ImageNumber                                                  
1            0.000000                  1   2  1.5  0.707107  
2            0.000000                  1   2  1.5  0.707107  
3            0.000000                  1   1  1.0  0.000000  
4            0.577350                  1   3  2.0  1.000000  
5            1.154701                  1   3  2.0  1.000000  

[5 rows x 253 columns]

In [55]:
filename = YNEG_DIR+CELLS
df_cneg = make_dataframe(filename) 
df_cneg = shave_objects(df_cneg,"Cell_")
df_cneg.head()

Cell_ObjectNumber Cell_AreaShape_Area                              \
                          max                 min    max    mean          std   
ImageNumber                                                                     
1                           2                3848  10157  7002.5  4461.136683   
2                           2                4076   4221  4148.5   102.530483   
3                           1                7399   7399  7399.0     0.000000   
4                           6                1692   7259  4792.0  1911.639087   
5                           2                3045   5803  4424.0  1950.200503   

            Cell_AreaShape_BoundingBoxArea                                    \
                                       min    max          mean          std   
ImageNumber                                                                    
1                                     7654  20943  14298.500000  9396.742015   
2                                     7192   7350   7271.000000   111.722871   
3                                    25050  25050  25050.000000     0.000000   
4                                     3843  16999   9323.666667  4527.963987   
5                                     5964   8532   7248.000000  1815.850214   

            Cell_AreaShape_BoundingBoxMaximum_X  ...  \
                                            min  ...   
ImageNumber                                      ...   
1                                           179  ...   
2                                           192  ...   
3                                           150  ...   
4                                            93  ...   
5                                            71  ...   

            Cell_Neighbors_SecondClosestDistance_Expanded             \
                                                     mean        std   
ImageNumber                                                            
1                                               62.908555  20.415496   
2                                               58.823873   1.073175   
3                                               86.229242   0.000000   
4                                               64.954607  18.807285   
5                                               61.695150  14.772483   

            Cell_Neighbors_SecondClosestObjectNumber_Expanded           \
                                                          min max mean   
ImageNumber                                                              
1                                                           0   0  0.0   
2                                                           0   0  0.0   
3                                                           0   0  0.0   
4                                                           2   6  3.5   
5                                                           0   0  0.0   

                      Cell_Parent_Nuclei                     
                  std                min max mean       std  
ImageNumber                                                  
1            0.000000                  1   2  1.5  0.707107  
2            0.000000                  1   2  1.5  0.707107  
3            0.000000                  1   1  1.0  0.000000  
4            1.378405                  1   6  3.5  1.870829  
5            0.000000                  1   2  1.5  0.707107  

[5 rows x 253 columns]

Merge documentation at [pandas](https://pandas.pydata.org/docs/dev/user_guide/merging.html).



In [ ]:
combine = pd.concat([df_ipos, df_rpos], axis=1, join="inner")
combine.head()